In [0]:
# 05_gold_kpis (global_temp only)\nfrom perf_lab_utils import perf_lab\nfrom pyspark.sql import functions as F\n\nwide = spark.table("global_temp.sales_wide_silver")\nprint("Input rows (sales_wide_silver):", wide.count())\n\n# Item-Level -> Order-Level aggregieren, damit KPIs nicht doppelt gezaehlt werden\norder_level = (\n    wide.groupBy("order_id", "customer_id")\n        .agg(\n            F.max("order_ts").alias("order_ts"),\n            F.max("customer_country").alias("customer_country"),\n            F.sum(F.coalesce(F.col("line_amount"), F.lit(0.0))).alias("order_revenue"),\n            F.sum(F.coalesce(F.col("quantity"), F.lit(0))).alias("units")\n        )\n)\n\ngold_kpis = (\n    order_level\n        .agg(\n            F.countDistinct("order_id").alias("orders_cnt"),\n            F.countDistinct("customer_id").alias("customers_cnt"),\n            F.round(F.sum("order_revenue"), 2).alias("gross_revenue"),\n            F.round(F.avg("order_revenue"), 2).alias("avg_order_value"),\n            F.round(F.sum("units"), 2).alias("units_sold")\n        )\n        .withColumn("kpi_ts", F.current_timestamp())\n)\n\nperf_lab(gold_kpis, "gold_kpis", action="count", emit_view="perf_metrics")\ngold_kpis.createOrReplaceGlobalTempView("gold_kpis")\n\nprint("Created view: global_temp.gold_kpis")\ndisplay(gold_kpis)\n